<a href="https://colab.research.google.com/github/Kezia-Hector/Exchange_Coefficient/blob/main/Avging_All_Basin_Hurricanes_Upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install netCDF4
!pip install scipy
import netCDF4 as nc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import netCDF4 as nc
import numpy as np
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

In [ ]:
TC_data_dir = '/content/drive/MyDrive/Colab Notebooks/tcdata_netcdf/'#Shared with me/MCFC IFs/Value Chain Resilience/Resiliency UROP/Kezia'
basins = ['shtracks','wptracks','attracks','iotracks','eptracks']

In [ ]:
errorstorms = []
k=0

In [ ]:
for basin in basins:
  fn = TC_data_dir+basin+'.nc'
  ds = nc.Dataset(fn)
  # get list of storm names
  sn_data= ds.variables['stnamec'][:]
  N_chars_per_name = sn_data.shape[0]
  N_dat_pts = sn_data.shape[1]
  storm_names = []
  for i in range(N_dat_pts):
    name_bytes = sn_data.transpose()[i]
    name_string = name_bytes.tobytes().decode("utf-8")
    storm_names.append(name_string)
  #print(storm_names)
  fn = TC_data_dir+basin+'.nc'
  ds = nc.Dataset(fn)
      # see what variables are there
      #print(ds.variables.keys())
      # look at a specific variable
  lat_data= ds.variables['latmc']
      # get the data out of the variable
  lat = lat_data[:]
  lon= ds.variables['longmc'][:] # longitude
  month= ds.variables['monthmc'][:] # month
  day= ds.variables['daymc'][:] # day
  hour= ds.variables['hourmc'][:] # hour
  v= ds.variables['vsmc'][:] # speed m/s
  year = ds.variables["yearic"][:]
  #Database only has from 1959 forward so change the startpt to 1959 rather than the beginning of the dataset
  startpt = np.argwhere(np.array(year).transpose() >= 1959)
  updated_storm_names = storm_names[startpt[0][0]:]
  #Storms we have already looked at
  Done_Storm_Names = ['Not_named   ']
  if 'Not_named' in storm_names[0]:
    Done_Storm_Names.append(storm_names[0])
  
  #R value using Kerry data
  R_Kerry = {}

  max_v_dict = {}

  for char in updated_storm_names:
    if char not in Done_Storm_Names:
      match = [i for i, s in enumerate(updated_storm_names) if char in s]
      n = len(match)
      i=0
      Done_Storm_Names.append(char)
      while i< n:
        fn_Kerry = TC_data_dir+basin+'.nc'
        ds_Kerry = nc.Dataset(fn_Kerry)
        # see what variables are there
        #print(ds.variables.keys())
        # look at a specific variable
        lat_data= ds_Kerry.variables['latmc']
        # get the data out of the variable
        lat = lat_data[:]
        lon= ds_Kerry.variables['longmc'][:] # longitude
        month= ds_Kerry.variables['monthmc'][:] # month
        day= ds_Kerry.variables['daymc'][:] # day
        hour= ds_Kerry.variables['hourmc'][:] # hour
        v= ds_Kerry.variables['vsmc'][:] # speed m/s
        year = ds_Kerry.variables["yearic"][:]
        Storm_number = match[i]
        Storm_lat_array = np.array(lat).transpose()[Storm_number + startpt[0][0]][:]
        Storm_lon_array = np.array(lon).transpose()[Storm_number + startpt[0][0]][:]
        Storm_month_array = np.array(month).transpose()[Storm_number + startpt[0][0]][:]
        Storm_day_array = np.array(day).transpose()[Storm_number + startpt[0][0]][:]
        Storm_hour_array = np.array(hour).transpose()[Storm_number + startpt[0][0]][:]
        Storm_v_array = np.array(v).transpose()[Storm_number + startpt[0][0]][:]
        Storm_year = np.array(year).transpose()[Storm_number + startpt[0][0]]
        if Storm_year < 1959.0:
          i+=1
          continue
        else:
          max_v = max(Storm_v_array)
          max_v_location = np.argwhere(Storm_v_array == max_v)[0][0]
          max_v_hour = Storm_hour_array[max_v_location]
          max_v_day = Storm_day_array[max_v_location]
          max_v_month = Storm_month_array[max_v_location]

          try:         
            import math
            import numpy as np
            fn_cds_p = '/content/drive/MyDrive/Colab Notebooks/{0} Hurricane Data/{1}Pressure.nc'.format(basin,char+str(Storm_year))

            fn_cds_ee = '/content/drive/MyDrive/Colab Notebooks/{0} Hurricane Data/{1}EverythingElse.nc'.format(basin,char+str(Storm_year))

            ds_cds_p = nc.Dataset(fn_cds_p)
            ds_cds_ee = nc.Dataset(fn_cds_ee)

            u0 = ds_cds_ee['u10'][:]
            v0 = ds_cds_ee['v10'][:]
            Td0 = ds_cds_ee['d2m'][:]
            Ta0 = ds_cds_ee['t2m'][:]
            Ts0 = ds_cds_ee['sst'][:]
            p0 = ds_cds_ee['sp'][:]
            To0 = ds_cds_p['t'][:]
            latitude = ds_cds_ee['latitude'][:]
            longitude = ds_cds_ee['longitude'][:]



            Lv = 2260000.0
            cp = 4186.0
            a = 17.27 # for Td to q calc
            b = 35.86 # for Td to q calc
            
            max_v_lat = Storm_lat_array[max_v_location]
            max_v_lat_round = round(max_v_lat*4)/4


            max_v_lon = Storm_lon_array[max_v_location]
            max_v_lon_round = round(max_v_lon*4)/4

            latitude_start = np.argwhere(latitude == max_v_lat_round - 0.5)[0][0]
            latitude_end = np.argwhere(latitude == max_v_lat_round + 0.5)[0][0]

            longitude_end = np.argwhere(longitude == max_v_lon_round + 0.5)[0][0]
            longitude_start = np.argwhere(longitude == max_v_lon_round - 0.5)[0][0]
            
            if latitude_end < latitude_start and longitude_start < longitude_end:
              Td1 = Td0[0,latitude_end:latitude_start,longitude_start:longitude_end]
              Ta1 = Ta0[0,latitude_end:latitude_start,longitude_start:longitude_end]
              To1 = To0[0,latitude_end:latitude_start,longitude_start:longitude_end]
              Ts1 = Ts0[0,latitude_end:latitude_start,longitude_start:longitude_end]
              p1 = p0[0,latitude_end:latitude_start,longitude_start:longitude_end]
            if latitude_end > latitude_start and longitude_start < longitude_end:
              Td1 = Td0[0,latitude_start:latitude_end,longitude_start:longitude_end]
              Ta1 = Ta0[0,latitude_start:latitude_end,longitude_start:longitude_end]
              To1 = To0[0,latitude_start:latitude_end,longitude_start:longitude_end]
              Ts1 = Ts0[0,latitude_start:latitude_end,longitude_start:longitude_end]
              p1 = p0[0,latitude_start:latitude_end,longitude_start:longitude_end]
            if latitude_end < latitude_start and longitude_start > longitude_end:
              Td1 = Td0[0,latitude_end:latitude_start,longitude_end:longitude_start]
              Ta1 = Ta0[0,latitude_end:latitude_start,longitude_end:longitude_start]
              To1 = To0[0,latitude_end:latitude_start,longitude_end:longitude_start]
              Ts1 = Ts0[0,latitude_end:latitude_start,longitude_end:longitude_start]
              p1 = p0[0,latitude_end:latitude_start,longitude_end:longitude_start]
            if latitude_end > latitude_start and longitude_start > longitude_end:
              Td1 = Td0[0,latitude_start:latitude_end,longitude_end:longitude_start]
              Ta1 = Ta0[0,latitude_start:latitude_end,longitude_end:longitude_start]
              To1 = To0[0,latitude_start:latitude_end,longitude_end:longitude_start]
              Ts1 = Ts0[0,latitude_start:latitude_end,longitude_end:longitude_start]
              p1 = p0[0,latitude_start:latitude_end,longitude_end:longitude_start]


            Td2 = Td1[Td1>0]
            Ta2 = Ta1[Ta1>0]
            To2 = To1[To1>0]
            Ts2 = Ts1[Ts1>0]
            p2 = p1[p1>0]


            Td = np.nanmean(Td2)
            Ta = np.nanmean(Ta2)
            Ts = np.nanmean(Ts2)
            To = np.nanmean(To2)
            p = np.nanmean(p2)

            e = 6.1078*math.exp(a*(Td-273.16)/(Td-b))
            qa = 0.622*(e)/(p/100.0-0.378*(e))
            es = 6.1078*math.exp(a*(Ts-273.16)/(Ts-b))
            qs = 0.622*(es)/(p/100.0-0.378*(es))

            #K = k* -k
            K= Lv * (qs - qa) + cp*(Ts -Ta)
            

            Name = basin + char + str(Storm_year)
            

            #Ratio Calculated using data from Kerry
            R_Kerry[Name] = (max_v*max_v * To/(Ts-To))/K 

        
            max_v_dict[Name] = max_v
            k+=1
            if k%100 == 0:
              print(k)
          except:
            test = Storm_v_array[Storm_v_array!=0]
            if len(test) > 0:
              errorstorms.append(basin+char+str(Storm_year))
          finally:
            i+=1
      
  #Putting Evertything for this basin in the google drive
  gc = gspread.authorize(creds)
  sh = gc.create('{0} FINAL'.format(basin))

  # Open our new sheet and add some data.
  worksheet = gc.open('{0} FINAL'.format(basin)).sheet1

  names = R_Kerry.keys()
  ratios = R_Kerry.values()
  maxvs = max_v_dict.values()

  z = len(names)

  cell_list = worksheet.range('A1:A{0}'.format(z))

  i = 0

  for cell in cell_list:
    cell.value = list(names)[i]
    i+=1
  worksheet.update_cells(cell_list)

  cell_list = worksheet.range('B1:B{0}'.format(z))
  i = 0

  for cell in cell_list:
    cell.value = list(ratios)[i]
    i+=1
  worksheet.update_cells(cell_list)

  cell_list = worksheet.range('C1:C{0}'.format(z))

  i = 0

  for cell in cell_list:
    cell.value = list(maxvs)[i]
    i+=1
  worksheet.update_cells(cell_list)
 
 
  print(errorstorms)
 